In [ ]:
import OpenDartReader
import FinanceDataReader as fdr
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import time

pd.set_option('display.max_columns', None)
path = '/Users/ljhee/Desktop/BOAZ/Fintshift/Data/'

## 1. Data Collection

### 1.1 Kospi & Kosdaq

In [ ]:
# KRX 데이터
kospi = pd.read_csv('data_1113_20230902.csv', encoding = 'cp949')
kosdaq = pd.read_csv('data_1137_20230902.csv', encoding = 'cp949')

df_code = pd.concat([kospi, kosdaq], axis = 0).reset_index(drop = True).loc[:, ['종목코드', '종목명', '시장구분', '업종명']]

### 1.2 OpenDartReader
- 연결재무제표, 별도재무재표 중 연결재무제표 선택
- 보고서 코드 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)
- 2016~2023년도 3분가까지의 데이터 수집 (2016년부터 데이터 제공)

In [7]:
# 본인 api 입력
my_api = "0f429235d11ce8816af3d06d8dd400016feb0251"
dart = OpenDartReader(my_api)

year_lst = []
for _year in ['2016','2017','2018','2019','2020','2021','2022']:
    for _ in range(4):
        year_lst.append(_year)
        
quarter_lst = []
for _ in range(7):
    for q in ['11013','11012','11014','11011']:
        quarter_lst.append(q)

In [ ]:
col = ['rcept_no', 'corp_code', 'stock_code', '당기순이익', '매출액_당기', '법인세차감전 순이익',
       '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채', '유동자산_당기', '이익잉여금', '자본금',
       '자본총계_당기', '자산총계_당기', '매출액_전기', '자산총계_전기', '유동자산_전기', '자본총계_전기', 'year', 'quarter']
df_fin_final = pd.DataFrame(columns = col)

for code in tqdm(df_code["종목코드"]):
    time.sleep(1)
    for i in range(8):
        try:
            data_fin = dart .finstate(code, year_lst[i],reprt_code = quarter_lst[i])
            #연결재무제표 있으면 무조건 연결재무제표로, 연결재무제표 없으면 재무제표
            if '연결재무제표' in list(data_fin['fs_nm'].unique()):
                data_fin = data_fin[data_fin["fs_nm"] == "연결재무제표"]
            else:
                data_fin = data_fin[data_fin["fs_nm"] == "재무제표"]
            #필요한 컬럼만 인덱싱 (기존코드랑 동일)
            #data_fin1 -> 당기, data_fin2 -> 전기 데이터 용도
            data_fin = data_fin.loc[:, ["rcept_no", "corp_code", "stock_code", "account_nm", 
                            "thstrm_amount", "frmtrm_amount"]].reset_index(drop = True)
            data_fin1 = data_fin.pivot(["rcept_no", "corp_code", "stock_code"],
            "account_nm", "thstrm_amount").reset_index(level=[0,1,2]).rename_axis(None, axis=1)

            data_fin2 = data_fin.pivot("stock_code","account_nm","frmtrm_amount"
                                      ).reset_index(level=0).rename_axis(None, axis=1)
            #data_fin2 인덱싱 할 때 '매출액', '자산총계', '유동자산', '자본총계' 중 특히 매출액 데이터가 없어서, 매출액으로 인덱싱을 하려고 할 때 오류 발생
            #그래서 4개 데이터 중 없으면 그냥 None 값을 넣도록 코드 수정하여 인덱스 에러 방지
            if all(_col in data_fin2.columns for _col in ['stock_code', '매출액', '자산총계', '유동자산', '자본총계']):
                data_fin2 = data_fin2.loc[:,['stock_code', '매출액', '자산총계', '유동자산', '자본총계']]
            else:
                # 필수 컬럼 중 일부 또는 전체가 존재하지 않는 경우에는 null 값을 할당하여 컬럼을 그대로 유지
                for _col in ['stock_code', '매출액', '자산총계', '유동자산', '자본총계']:
                    if _col not in data_fin2.columns:
                        data_fin2[_col] = None
                        data_fin1[_col] = None
                data_fin2 = data_fin2.loc[:,['stock_code', '매출액', '자산총계', '유동자산', '자본총계']]

            df_fin = pd.merge(data_fin1, data_fin2, on = 'stock_code', how = 'inner', suffixes = ['_당기', '_전기'])
            df_fin["year"] = year_lst[i]
            df_fin["quarter"] = quarter_lst[i]
            
            #여기 코드는 우리가 원하는 컬림 구성인 'col' 변수로 최종 데이터 프레임 인덱싱 하는 과정인데, 여기도 혹시 우리가 지정한 필수 컬럼 중 데이터가 없는 경우도 분명히 존재하기에
            #인덱싱 오류 안나게 코드 수정
            if all(_col in df_fin.columns for _col in col):
                df_fin = df_fin.loc[:,col]
                df_fin_final = df_fin_final.append(df_fin)
            else:
                # 필수 컬럼 중 일부 또는 전체가 존재하지 않는 경우에는 null 값을 할당하여 컬럼을 그대로 유지
                for _col in col:
                    if _col not in df_fin.columns:
                        df_fin[_col] = None
                df_fin = df_fin.loc[:,col]
                df_fin_final = df_fin_final.append(df_fin)

        except:
            row = [np.NaN]* len(col)
            row[2] = code # stock_code
            row[20] = year_lst[i] # year
            row[21] = quarter_lst[i] # quarter
            df_fin_final = df_fin_final.append(pd.Series(row, index=df_fin_final.columns), ignore_index=True)

In [ ]:
df_fin_final.reset_index(drop=True,inplace=True)

df_fin_final = df_fin_final[['stock_code', 'year','quarter', 'rcept_no', 'corp_code', '당기순이익', '매출액_당기', '법인세차감전 순이익',
       '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채', '유동자산_당기', '이익잉여금', '자본금',
       '자본총계_당기', '자산총계_당기', '매출액_전기', '자산총계_전기', '유동자산_전기', '자본총계_전기']]

### 기존에 수집했던 데이터를 활용하여 id, distb_stock_code 컬럼을 추가 ###
ndf = pd.read_excel(f"{path}/1_dartdata_concat_1523.xlsx")

ndf = ndf.loc[:,['stock_code','year','quarter','id','distb_stock_co']]
df_merge = pd.merge(df_fin_final,ndf,on = ['stock_code','year','quarter'], how='left')

df_merge['year'] = df_merge['year'].astype('int')
df_merge['quarter'] = df_merge['quarter'].astype('int')

df_merge = df_merge.sort_values(by=['stock_code','year','quarter'],axis=0)
df_merge.reset_index(drop=True,inplace=True)

df_merge.to_excel('1_dartdata_concat_1523_refine.xlsx',index=False)

### 1.3 Stock Price Data : FinanceAPI

In [ ]:
reprt_code_lst = list(df_merge['stock_code'].unique())

stock_lst = []
start_lst = []; end_lst = []

for x in ['2016','2017','2018','2019','2020','2021']:
    for y in ['01','04','07','10']:
        start_lst.append(x+'-'+y+'-01')
    for z in ['03']:
        end_lst.append(x+'-'+z+'-31')
    for a in ['06','09']:
        end_lst.append(x+'-'+a+'-30')
    for z in ['12']:
        end_lst.append(x+'-'+z+'-31')
start_lst = start_lst[:-2]
end_lst = end_lst[:-2]

stock_lst = []; stock_name = []

final_df = pd.DataFrame()

for _stock in tqdm(reprt_code_lst):
    stock = fdr.DataReader(_stock, start='2016-01-01', end='2021-06-30')
    stock = stock.reset_index()
    stock['stock'] = _stock
    final_df = pd.concat([final_df,stock],axis=0)
    
final_df.to_csv('16_23_주식가격.csv',index=False)

## 2. Data Preprocessing for Visualization

### 2-1. Concat

### 1_dartdata_concat_1215 : 4분기 고려한 raw 데이터 합친 데이터

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Fintshift/Data&Code/ForViz/데이터_수정본_1130/1_dartdata_concat_1523_refine.xlsx")

df_code1 = pd.read_csv(f"{path}/data_1137_20230902.csv", encoding = "cp949")
df_code2 = pd.read_csv(f"{path}/data_1113_20230902.csv", encoding = "cp949")
df_code = pd.concat([df_code1, df_code2], axis = 0).reset_index(drop = True).loc[:, ['종목코드', '종목명', '시장구분', '업종명']]

print(df.shape)
df.head(2)

(82304, 24)


,stock_code,year,quarter,rcept_no,corp_code,당기순이익,매출액_당기,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산_당기,이익잉여금,자본금,자본총계_당기,자산총계_당기,매출액_전기,자산총계_전기,유동자산_전기,자본총계_전기,id,distb_stock_co
0,000020,2015,11011,2.016033e+13,119195.0,"5,608,652,157","223,201,285,434","6,000,622,879","87,069,287,627","17,920,239,528","182,172,344,113","4,812,973,681","69,149,048,099","135,014,685,939","174,195,360,608","27,931,470,000","230,117,742,425","317,187,030,052","213,472,144,940","313,073,099,664","120,547,703,134","229,725,782,416",000020_2015_11011,27931470
1,000020,2016,11011,2.017041e+13,119195.0,"26,254,318,411","237,470,834,801","35,655,076,190","71,679,236,748","16,227,160,789","173,112,688,778","11,259,333,902","55,452,075,959","151,491,847,872","197,195,919,026","27,931,470,000","252,925,299,902","324,604,536,650","223,201,285,434","317,187,030,052","135,014,685,939","230,117,742,425",000020_2016_11011,27931470


In [ ]:
df.columns

Index(['stock_code', 'year', 'quarter', 'rcept_no', 'corp_code', '당기순이익',
       '매출액_당기', '법인세차감전 순이익', '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채',
       '유동자산_당기', '이익잉여금', '자본금', '자본총계_당기', '자산총계_당기', '매출액_전기', '자산총계_전기',
       '유동자산_전기', '자본총계_전기', 'id', 'distb_stock_co'],
      dtype='object')

In [ ]:
# float으로 type 변환
odj_li = [ '당기순이익','매출액_당기', '법인세차감전 순이익', '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채',
       '유동자산_당기', '이익잉여금', '자본금', '자본총계_당기', '자산총계_당기', '매출액_전기', '자산총계_전기',
       '유동자산_전기', '자본총계_전기', 'distb_stock_co']
def tofloat(value):
    if (type(value) == float) |(type(value) == int):
        return value
    elif value == '-':
        return 0
    else:
        return float(value.replace(',', ''))

for col in odj_li:
    df[col] = df[col].apply(tofloat)

In [ ]:
df.loc[df['quarter'] == 11013, 'quarter'] = '1분기'
df.loc[df['quarter'] == 11012, 'quarter'] = '2분기'
df.loc[df['quarter'] == 11014, 'quarter'] = '3분기'
df.loc[df['quarter'] == 11011, 'quarter'] = '4분기'

In [ ]:
df['time'] = df['year'].astype(str) + '_' + df['quarter'].astype(str)

In [ ]:
df = df[df["time"] != "2015_4분기"].reset_index(drop = True)
df.shape

(79732, 25)

In [ ]:
df_melt = df.loc[:,['stock_code', '당기순이익',
       '매출액_당기', '법인세차감전 순이익', '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채',
       '유동자산_당기', '이익잉여금', '자본금', '자본총계_당기', '자산총계_당기', '매출액_전기', '자산총계_전기',
       '유동자산_전기', '자본총계_전기','distb_stock_co', 'time']].melt(id_vars = ['stock_code', 'time'])
df_melt

,stock_code,time,variable,value
0,000020,2016_4분기,당기순이익,2.625432e+10
1,000020,2016_2분기,당기순이익,9.782102e+08
2,000020,2016_1분기,당기순이익,4.785023e+09
3,000020,2016_3분기,당기순이익,1.437847e+09
4,000020,2017_4분기,당기순이익,4.700901e+10
...,...,...,...,...
1435171,950220,2022_1분기,distb_stock_co,1.974830e+07
1435172,950220,2022_3분기,distb_stock_co,1.976549e+07
1435173,950220,2023_2분기,distb_stock_co,1.977349e+07
1435174,950220,2023_1분기,distb_stock_co,1.976549e+07


In [ ]:
df_pivot = df_melt.pivot(index = ['stock_code', 'variable'], columns = "time", values = 'value').reset_index()
df_pivot = df_pivot.rename_axis(None, axis=1)
df_pivot

,stock_code,variable,2016_1분기,2016_2분기,2016_3분기,2016_4분기,2017_1분기,2017_2분기,2017_3분기,2017_4분기,2018_1분기,2018_2분기,2018_3분기,2018_4분기,2019_1분기,2019_2분기,2019_3분기,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기,2022_2분기,2022_3분기,2022_4분기,2023_1분기,2023_2분기,2023_3분기
0,000020,distb_stock_co,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,2.793147e+07,NaN
1,000020,당기순이익,4.785023e+09,9.782102e+08,1.437847e+09,2.625432e+10,3.429906e+09,3.187634e+09,7.839167e+08,4.700901e+10,4.711062e+09,7.499043e+08,1.177560e+09,1.006811e+10,1.841873e+09,-6.310758e+08,2.334355e+08,9.393855e+09,2.259000e+09,6.719645e+09,1.175514e+10,2.871510e+10,5.510603e+09,7.039931e+09,7.879967e+08,1.958280e+10,5.801167e+09,6.471107e+09,6.843468e+09,2.159121e+10,1.293368e+10,7.580363e+09,6.157136e+09
2,000020,매출액_당기,5.923046e+10,6.251293e+10,5.736306e+10,2.374708e+11,6.064106e+10,6.695455e+10,6.443305e+10,2.588816e+11,7.696418e+10,7.933752e+10,7.491474e+10,3.066026e+11,7.466335e+10,7.492866e+10,7.421699e+10,3.071500e+11,6.700367e+10,6.983330e+10,6.656821e+10,2.720754e+11,7.182724e+10,7.580235e+10,6.907114e+10,2.930181e+11,8.529413e+10,8.721816e+10,8.345552e+10,3.404263e+11,9.941981e+10,8.999036e+10,8.745741e+10
3,000020,매출액_전기,5.240362e+10,5.728237e+10,5.585974e+10,2.232013e+11,5.923046e+10,6.251293e+10,5.736306e+10,2.374708e+11,6.064106e+10,6.695455e+10,6.443305e+10,2.588816e+11,7.696418e+10,7.933752e+10,7.491474e+10,3.066026e+11,7.466335e+10,7.492866e+10,7.421699e+10,3.071500e+11,6.700367e+10,6.983330e+10,6.656821e+10,2.720754e+11,7.182724e+10,7.580235e+10,6.907114e+10,2.930181e+11,8.529413e+10,8.721816e+10,8.345552e+10
4,000020,법인세차감전 순이익,6.748348e+09,2.218524e+09,2.314823e+09,3.565508e+10,4.931641e+09,4.567346e+09,1.500644e+09,6.521874e+10,6.212383e+09,1.648785e+09,1.978956e+09,1.453906e+10,2.834298e+09,-4.172020e+08,7.070189e+08,1.660787e+10,3.268501e+09,7.070109e+09,1.585186e+10,3.363828e+10,7.688558e+09,9.739095e+09,2.261543e+09,2.652729e+10,7.640960e+09,9.035201e+09,9.450743e+09,2.308555e+10,1.694877e+10,1.016415e+10,8.735933e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46291,950220,자본금,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.655000e+03,1.968000e+03,1.968000e+03,1.968000e+03,1.972000e+03,1.975000e+03,1.975000e+03,1.977000e+03,1.977000e+03,1.977000e+03,1.978000e+03,1.978000e+03
46292,950220,자본총계_당기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.741423e+07,1.657324e+08,1.560762e+08,1.472060e+08,1.366870e+08,1.263794e+08,1.089918e+08,9.383146e+07,9.444430e+07,8.675882e+07,7.525099e+07,6.587899e+07
46293,950220,자본총계_전기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000914e+08,7.741423e+07,7.741423e+07,7.741423e+07,7.741423e+07,1.366870e+08,1.366870e+08,1.366870e+08,1.366870e+08,9.444430e+07,9.444430e+07,9.444430e+07
46294,950220,자산총계_당기,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.790894e+07,1.800134e+08,1.697881e+08,1.635438e+08,1.577625e+08,1.500472e+08,1.354946e+08,1.206298e+08,1.103649e+08,9.830879e+07,8.711068e+07,7.710694e+07


In [ ]:
df_pivot.isna().sum()

stock_code        0
variable          0
2016_1분기      18469
2016_2분기      17772
2016_3분기      17824
2016_4분기      16818
2017_1분기      17039
2017_2분기      16273
2017_3분기      14555
2017_4분기      13795
2018_1분기      12778
2018_2분기      12631
2018_3분기      12325
2018_4분기      11262
2019_1분기      11336
2019_2분기      11143
2019_3분기      10721
2019_4분기       9842
2020_1분기       9834
2020_2분기       9603
2020_3분기       9229
2020_4분기       7933
2021_1분기       8055
2021_2분기       7622
2021_3분기       7230
2021_4분기       6243
2022_1분기       6389
2022_2분기       5974
2022_3분기       5768
2022_4분기       4776
2023_1분기       5188
2023_2분기       4943
2023_3분기       6990
dtype: int64

In [ ]:
df_pivot["2016_4분기"] = df_pivot["2016_4분기"] - df_pivot["2016_3분기"] - df_pivot["2016_2분기"]- df_pivot["2016_1분기"]
df_pivot["2017_4분기"] = df_pivot["2017_4분기"] - df_pivot["2017_3분기"] - df_pivot["2017_2분기"]- df_pivot["2017_1분기"]
df_pivot["2018_4분기"] = df_pivot["2018_4분기"] - df_pivot["2018_3분기"] - df_pivot["2018_2분기"]- df_pivot["2018_1분기"]
df_pivot["2019_4분기"] = df_pivot["2019_4분기"] - df_pivot["2019_3분기"] - df_pivot["2019_2분기"]- df_pivot["2019_1분기"]
df_pivot["2020_4분기"] = df_pivot["2020_4분기"] - df_pivot["2020_3분기"] - df_pivot["2020_2분기"]- df_pivot["2020_1분기"]
df_pivot["2021_4분기"] = df_pivot["2021_4분기"] - df_pivot["2021_3분기"] - df_pivot["2021_2분기"]- df_pivot["2021_1분기"]
df_pivot["2022_4분기"] = df_pivot["2022_4분기"] - df_pivot["2022_3분기"] - df_pivot["2022_2분기"]- df_pivot["2022_1분기"]

In [ ]:
df_pivot = df_pivot.rename(columns = {"variable":"지표"})
df_melt2 = df_pivot.melt(id_vars = ['stock_code', '지표'])
df_melt2

,stock_code,지표,variable,value
0,000020,distb_stock_co,2016_1분기,2.793147e+07
1,000020,당기순이익,2016_1분기,4.785023e+09
2,000020,매출액_당기,2016_1분기,5.923046e+10
3,000020,매출액_전기,2016_1분기,5.240362e+10
4,000020,법인세차감전 순이익,2016_1분기,6.748348e+09
...,...,...,...,...
1435171,950220,자본금,2023_3분기,1.978000e+03
1435172,950220,자본총계_당기,2023_3분기,6.587899e+07
1435173,950220,자본총계_전기,2023_3분기,9.444430e+07
1435174,950220,자산총계_당기,2023_3분기,7.710694e+07


In [ ]:
df_pivot2 = df_melt2.pivot(index = ['stock_code', 'variable'], columns = "지표", values = 'value').reset_index()
df_pivot2 = df_pivot2.rename_axis(None, axis=1)
df_pivot2

,stock_code,variable,distb_stock_co,당기순이익,매출액_당기,매출액_전기,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산_당기,유동자산_전기,이익잉여금,자본금,자본총계_당기,자본총계_전기,자산총계_당기,자산총계_전기
0,000020,2016_1분기,27931470.0,4.785023e+09,5.923046e+10,5.240362e+10,6.748348e+09,9.037785e+10,1.929455e+10,1.825681e+11,6.563294e+09,7.108330e+10,1.417109e+11,1.350147e+11,1.767672e+11,2.793147e+10,2.339012e+11,2.301177e+11,3.242790e+11,3.171870e+11
1,000020,2016_2분기,27931470.0,9.782102e+08,6.251293e+10,5.728237e+10,2.218524e+09,9.664252e+10,2.045581e+10,1.809665e+11,2.109206e+09,7.618671e+10,1.499458e+11,1.350147e+11,1.777454e+11,2.793147e+10,2.342698e+11,2.301177e+11,3.309123e+11,3.171870e+11
2,000020,2016_3분기,27931470.0,1.437847e+09,5.736306e+10,5.585974e+10,2.314823e+09,9.323437e+10,2.165487e+10,1.787469e+11,2.081179e+09,7.157950e+10,1.496770e+11,1.350147e+11,1.791833e+11,2.793147e+10,2.351896e+11,2.301177e+11,3.284239e+11,3.171870e+11
3,000020,2016_4분기,-55862940.0,1.905324e+10,5.836439e+10,5.765556e+10,2.437338e+10,-2.085755e+11,-4.517807e+10,-3.691689e+11,5.056553e+08,-1.633974e+11,-2.898419e+11,-2.700294e+11,-3.365000e+11,-5.586294e+10,-4.504352e+11,-4.602355e+11,-6.590107e+11,-6.343741e+11
4,000020,2017_1분기,27931470.0,3.429906e+09,6.064106e+10,5.923046e+10,4.931641e+09,6.926245e+10,1.731640e+10,1.705475e+11,4.655521e+09,5.194604e+10,1.519433e+11,1.514918e+11,1.975827e+11,2.793147e+10,2.532283e+11,2.529253e+11,3.224908e+11,3.246045e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79727,950220,2022_3분기,19765493.0,-1.531963e+07,0.000000e+00,0.000000e+00,-1.524875e+07,2.679834e+07,2.929607e+06,1.687295e+07,-1.268815e+07,2.386873e+07,1.037568e+08,1.406347e+08,-1.490843e+08,1.977000e+03,9.383146e+07,1.366870e+08,1.206298e+08,1.577625e+08
79728,950220,2022_4분기,-39496604.0,-2.055425e+06,0.000000e+00,0.000000e+00,-2.107109e+06,-6.104827e+07,-6.843749e+06,-3.739438e+07,-5.647321e+06,-5.420452e+07,-2.584123e+08,-2.812693e+08,2.480868e+08,-3.950000e+03,-2.347584e+08,-2.733740e+08,-2.958067e+08,-3.155249e+08
79729,950220,2023_1분기,19765493.0,-9.247988e+06,0.000000e+00,0.000000e+00,-9.201896e+06,1.154997e+07,2.615301e+06,1.727115e+07,-9.447804e+06,8.934665e+06,8.103763e+07,9.308837e+07,-1.595374e+08,1.977000e+03,8.675882e+07,9.444430e+07,9.830879e+07,1.103649e+08
79730,950220,2023_2분기,19773493.0,-1.272993e+07,0.000000e+00,0.000000e+00,-1.268167e+07,1.185969e+07,2.398690e+06,1.648801e+07,-1.285199e+07,9.460996e+06,7.062267e+07,9.308837e+07,-1.722456e+08,1.978000e+03,7.525099e+07,9.444430e+07,8.711068e+07,1.103649e+08


In [ ]:
df_final= pd.merge(df_pivot2, df_code,
                    left_on ="stock_code" , right_on = "종목코드", how = "left" )

In [ ]:
df_final = df_final.rename(columns = {"variable" : "time"})
df_final[['year', 'quarter']] = df_final['time'].str.split('_', expand=True)

In [ ]:
df_final.head(2)

,stock_code,time,distb_stock_co,당기순이익,매출액_당기,매출액_전기,법인세차감전 순이익,부채총계,비유동부채,비유동자산,영업이익,유동부채,유동자산_당기,유동자산_전기,이익잉여금,자본금,자본총계_당기,자본총계_전기,자산총계_당기,자산총계_전기,종목코드,종목명,시장구분,업종명,year,quarter
0,000020,2016_1분기,27931470.0,4.785023e+09,5.923046e+10,5.240362e+10,6.748348e+09,9.037785e+10,1.929455e+10,1.825681e+11,6.563294e+09,7.108330e+10,1.417109e+11,1.350147e+11,1.767672e+11,2.793147e+10,2.339012e+11,2.301177e+11,3.242790e+11,3.171870e+11,000020,동화약품,KOSPI,의약품,2016,1분기
1,000020,2016_2분기,27931470.0,9.782102e+08,6.251293e+10,5.728237e+10,2.218524e+09,9.664252e+10,2.045581e+10,1.809665e+11,2.109206e+09,7.618671e+10,1.499458e+11,1.350147e+11,1.777454e+11,2.793147e+10,2.342698e+11,2.301177e+11,3.309123e+11,3.171870e+11,000020,동화약품,KOSPI,의약품,2016,2분기


In [ ]:
df_final.to_excel(f"{path}/1_dartdata_concat_1215.xlsx", index = False)

### 2.2 Variable Data

### 2_dartdata_variable_1222 : 12 변수 만들기 (null 유지)

In [144]:
df = pd.read_excel(f"{path}/1_dartdata_concat_1215.xlsx")

In [145]:
# 수익성
df["영업이익률"] = df["영업이익"]/ df["매출액_당기"]*100
df["순이익률"] = df["당기순이익"]/ df["매출액_당기"]*100
df["ROE"] = df["당기순이익"]/ df["자본총계_당기"]*100

In [146]:
# 성장성
df["매출액증가율"] = (df["매출액_당기"]/ df["매출액_전기"]-1)*100
df["총자산증가율"] = (df["자산총계_당기"]/ df["자산총계_전기"]-1)*100
df["유동자산증가율"] = (df["유동자산_당기"]/ df["유동자산_전기"]-1)*100

In [147]:
# 안전성
df["부채비율"] = df["부채총계"]/ df["자본총계_당기"]*100
df["유동부채비율"] = df["유동부채"]/ df["자본총계_당기"]*100
df["비유동부채비율"] = df["비유동부채"]/ df["자본총계_당기"]*100

In [148]:
# 가치성
df["EPS"] = df["당기순이익"]/ df["distb_stock_co"]
df["BPS"] = df["자본총계_당기"]/ df["distb_stock_co"]
df["SPS"] = df["매출액_당기"]/ df["distb_stock_co"]

In [149]:
df.columns

Index(['stock_code', 'time', 'distb_stock_co', '당기순이익', '매출액_당기', '매출액_전기',
       '법인세차감전 순이익', '부채총계', '비유동부채', '비유동자산', '영업이익', '유동부채', '유동자산_당기',
       '유동자산_전기', '이익잉여금', '자본금', '자본총계_당기', '자본총계_전기', '자산총계_당기', '자산총계_전기',
       '종목코드', '종목명', '시장구분', '업종명', 'year', 'quarter', '영업이익률', '순이익률', 'ROE',
       '매출액증가율', '총자산증가율', '유동자산증가율', '부채비율', '유동부채비율', '비유동부채비율', 'EPS',
       'BPS', 'SPS'],
      dtype='object')

In [150]:
# df = df.fillna(0)
col_li = ['종목코드', '종목명', '시장구분', '업종명', "year", "quarter", "time", '영업이익률', '순이익률', 'ROE',
       '매출액증가율', '총자산증가율', '유동자산증가율', '부채비율', '유동부채비율', '비유동부채비율', 'EPS', 'BPS', 'SPS']
df_col = df.loc[:, col_li]
df_col = df_col.replace(float("inf"),np.nan).replace(float("-inf"), np.nan)

In [151]:
df_col

,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,11.080944,8.078652,2.045746,13.027416,2.235900,4.959643,38.639330,30.390312,8.249018,171.312963,8374.108836,2120.563589
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,3.374031,1.564813,0.417557,9.131184,4.327189,11.058866,41.252657,32.520926,8.731731,35.021794,8387.306372,2238.082339
2,000020,동화약품,KOSPI,의약품,2016,3분기,2016_3분기,3.628083,2.506573,0.611357,2.691244,3.542675,10.859787,39.642223,30.434811,9.207412,51.477671,8420.235831,2053.707069
3,000020,동화약품,KOSPI,의약품,2016,4분기,2016_4분기,0.866376,32.645314,-4.229962,1.229424,3.883618,7.337159,46.305325,36.275454,10.029871,-341.071170,8063.220995,-1044.778335
4,000020,동화약품,KOSPI,의약품,2017,1분기,2017_1분기,7.677175,5.656078,1.354472,2.381556,-0.651177,0.297977,27.351775,20.513519,6.838256,122.797196,9066.058440,2171.065993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79727,950220,네오이뮨텍,KOSDAQ,기타서비스,2022,3분기,2022_3분기,NaN,NaN,-16.326750,NaN,-23.537065,-26.222421,28.560076,25.437874,3.122201,-0.775069,4.747236,0.000000
79728,950220,네오이뮨텍,KOSDAQ,기타서비스,2022,4분기,2022_4분기,NaN,NaN,0.875549,NaN,-6.249331,-8.126375,26.004720,23.089490,2.915230,0.052041,5.943763,-0.000000
79729,950220,네오이뮨텍,KOSDAQ,기타서비스,2023,1분기,2023_1분기,NaN,NaN,-10.659421,NaN,-10.923844,-12.945486,13.312728,10.298279,3.014450,-0.467886,4.389408,0.000000
79730,950220,네오이뮨텍,KOSDAQ,기타서비스,2023,2분기,2023_2분기,NaN,NaN,-16.916627,NaN,-21.070288,-24.133740,15.760173,12.572587,3.187586,-0.643788,3.805650,0.000000


In [152]:
df_col.to_excel(f"{path}/2_dartdata_variable_1222.xlsx", index = False)

### 2.3 Scaling

### 3_dartdata_scaled_variable_1222 : 4분위수 스케일

In [105]:
df = pd.read_excel(f"{path}/2_dartdata_variable_1222.xlsx")

In [153]:
# df = df_col

In [154]:
df.describe()

,year,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS
count,79732.000000,5.946800e+04,5.901100e+04,60159.000000,5.940400e+04,60907.000000,6.081200e+04,60845.000000,60731.000000,60622.000000,5.148800e+04,5.185700e+04,5.122700e+04
mean,2019.387097,-1.166753e+02,-1.431313e+02,-0.202826,9.697401e+01,18.153105,-2.727807e+06,109.923083,79.112992,28.876661,-8.034417e+03,4.953398e+05,1.766933e+05
std,2.238640,9.327013e+03,1.314470e+04,129.124808,1.147985e+04,2006.468479,6.726821e+08,1000.096682,805.051548,360.606522,1.494904e+06,5.220351e+07,2.397103e+07
min,2016.000000,-1.955302e+06,-2.161272e+06,-28684.900507,-3.943715e+04,-925.316665,-1.658840e+11,-104759.590022,-39248.651886,-65510.938137,-2.638591e+08,-1.048225e+09,-9.744883e+08
25%,2017.000000,-2.105614e+00,-3.850804e+00,-1.021034,-9.047751e+00,-1.566778,-5.331168e+00,31.226272,21.466652,4.344077,-3.772671e+01,2.250529e+03,5.060899e+01
50%,2019.000000,3.667115e+00,2.813975e+00,0.865438,5.476624e+00,2.317485,2.706988e+00,68.180447,46.382110,13.875810,3.791546e+01,5.966684e+03,1.031652e+03
75%,2021.000000,9.204172e+00,8.745863e+00,2.763775,2.347177e+01,8.082078,1.276506e+01,129.662667,91.206498,35.257952,2.452353e+02,1.446920e+04,4.012620e+03
max,2023.000000,8.070565e+03,2.402449e+05,5658.176652,2.593148e+06,354030.780671,4.539778e+04,178554.362044,174785.721319,22123.282495,5.477574e+07,9.409741e+09,4.651584e+09


In [155]:
col_li = ['영업이익률','순이익률', 'ROE',
          '매출액증가율', '총자산증가율', '유동자산증가율', "BPS", "SPS", "EPS"]

for name in col_li:
  # df[f'{name}_score'] = pd.qcut(df[f'{name}'], q=[0, 0.25, 0.5, 0.75, 1], labels=[5, 10, 15, 20])
  q1 = df.loc[:, [f"{name}"]].quantile(0.25)[0]
  q2 = df.loc[:, [f"{name}"]].quantile(0.50)[0]
  q3 = df.loc[:, [f"{name}"]].quantile(0.75)[0]

  df[f'{name}_score'] = pd.cut(df[f'{name}'], bins=[-float('inf'), q1, q2, q3, float('inf')],
                               labels=[5, 10, 15, 20], include_lowest=True)
df.head(2)

,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS,영업이익률_score,순이익률_score,ROE_score,매출액증가율_score,총자산증가율_score,유동자산증가율_score,BPS_score,SPS_score,EPS_score
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,11.080944,8.078652,2.045746,13.027416,2.235900,4.959643,38.639330,30.390312,8.249018,171.312963,8374.108836,2120.563589,20,15,15,15,10,15,15,15,15
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,3.374031,1.564813,0.417557,9.131184,4.327189,11.058866,41.252657,32.520926,8.731731,35.021794,8387.306372,2238.082339,10,10,10,15,15,15,15,15,10


In [156]:
col_li = ['부채비율', '유동부채비율','비유동부채비율']

for name in col_li:
  # df[f'{name}_score'] = pd.qcut(df[f'{name}'], q=[0, 0.25, 0.5, 0.75, 1], labels=[5, 10, 15, 20])
  q1 = df.loc[:, [f"{name}"]].quantile(0.25)[0]
  q2 = df.loc[:, [f"{name}"]].quantile(0.50)[0]
  q3 = df.loc[:, [f"{name}"]].quantile(0.75)[0]

  df[f'{name}_score'] = pd.cut(df[f'{name}'], bins=[-float('inf'), q1, q2, q3, float('inf')],
                               labels=[20, 15, 10, 5], include_lowest=True)
df.head(2)

,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS,영업이익률_score,순이익률_score,ROE_score,매출액증가율_score,총자산증가율_score,유동자산증가율_score,BPS_score,SPS_score,EPS_score,부채비율_score,유동부채비율_score,비유동부채비율_score
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,11.080944,8.078652,2.045746,13.027416,2.235900,4.959643,38.639330,30.390312,8.249018,171.312963,8374.108836,2120.563589,20,15,15,15,10,15,15,15,15,15,15,15
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,3.374031,1.564813,0.417557,9.131184,4.327189,11.058866,41.252657,32.520926,8.731731,35.021794,8387.306372,2238.082339,10,10,10,15,15,15,15,15,10,15,15,15


In [7]:
df

,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS,영업이익률_score,순이익률_score,ROE_score,매출액증가율_score,총자산증가율_score,유동자산증가율_score,BPS_score,SPS_score,EPS_score,부채비율_score,유동부채비율_score,비유동부채비율_score
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,11.080944,8.078652,2.045746,13.027416,2.235900,4.959643,38.639330,30.390312,8.249018,171.312963,8374.108836,2120.563589,20,15,15,15,10,15,15,15,15,15,15,15
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,3.374031,1.564813,0.417557,9.131184,4.327189,11.058866,41.252657,32.520926,8.731731,35.021794,8387.306372,2238.082339,10,10,10,15,15,15,15,15,10,15,15,15
2,000020,동화약품,KOSPI,의약품,2016,3분기,2016_3분기,3.628083,2.506573,0.611357,2.691244,3.542675,10.859787,39.642223,30.434811,9.207412,51.477671,8420.235831,2053.707069,10,10,10,10,15,15,15,15,15,15,15,15
3,000020,동화약품,KOSPI,의약품,2016,4분기,2016_4분기,0.866376,32.645314,-4.229962,1.229424,3.883618,7.337159,46.305325,36.275454,10.029871,-341.071170,8063.220995,-1044.778335,10,20,5,10,15,15,15,5,5,15,15,15
4,000020,동화약품,KOSPI,의약품,2017,1분기,2017_1분기,7.677175,5.656078,1.354472,2.381556,-0.651177,0.297977,27.351775,20.513519,6.838256,122.797196,9066.058440,2171.065993,15,15,15,10,10,10,15,15,15,20,20,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79727,950220,네오이뮨텍,KOSDAQ,기타서비스,2022,3분기,2022_3분기,NaN,NaN,-16.326750,NaN,-23.537065,-26.222421,28.560076,25.437874,3.122201,-0.775069,4.747236,0.000000,NaN,NaN,5,NaN,5,5,5,5,10,20,15,20
79728,950220,네오이뮨텍,KOSDAQ,기타서비스,2022,4분기,2022_4분기,NaN,NaN,0.875549,NaN,-6.249331,-8.126375,26.004720,23.089490,2.915230,0.052041,5.943763,0.000000,NaN,NaN,15,NaN,5,5,5,5,10,20,15,20
79729,950220,네오이뮨텍,KOSDAQ,기타서비스,2023,1분기,2023_1분기,NaN,NaN,-10.659421,NaN,-10.923844,-12.945486,13.312728,10.298279,3.014450,-0.467886,4.389408,0.000000,NaN,NaN,5,NaN,5,5,5,5,10,20,20,20
79730,950220,네오이뮨텍,KOSDAQ,기타서비스,2023,2분기,2023_2분기,NaN,NaN,-16.916627,NaN,-21.070288,-24.133740,15.760173,12.572587,3.187586,-0.643788,3.805650,0.000000,NaN,NaN,5,NaN,5,5,5,5,10,20,20,20


In [157]:
df.to_excel(f"{path}/3_dartdata_scaled_variable_1222.xlsx", index = False)

In [ ]:
# df['SPS_score'] = pd.cut(df['SPS'], bins=[-float('inf'), 0,
#                                               df[df["SPS"] > 0]["SPS"].quantile(0.33),
#                                               df[df["SPS"] > 0]["SPS"].quantile(0.66), float('inf')],
#                                labels=[5, 10, 15, 20], include_lowest=True)
# df['EPS_score'] = pd.cut(df['EPS'], bins=[-float('inf'), 0,
#                                               df[df["EPS"] > 0]["EPS"].quantile(0.33),
#                                               df[df["EPS"] > 0]["EPS"].quantile(0.66), float('inf')],
#                                labels=[5, 10, 15, 20], include_lowest=True)
# df.head(2)

### 3_1_dartdata_stdscaled_variable_1222 :std 스케일링 (추후 동점자 보정을 위해)

In [158]:
df = pd.read_excel(f"{path}/2_dartdata_variable_1222.xlsx")

In [159]:
# Standardization 평균 0 / 분산 1
scaler = StandardScaler()

scaled = scaler.fit_transform(df.iloc[:, 7:])
df_st = pd.DataFrame(scaled, columns=df.columns[7:])
df_scale = pd.concat([df.iloc[:, :7], df_st], axis = 1)
df_scale.head(2)

,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,0.013698,0.011504,0.017414,-0.007313,-0.007933,0.004055,-0.071277,-0.060522,-0.057203,0.005489,-0.009328,-0.007283
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,0.012871,0.011008,0.004805,-0.007652,-0.006891,0.004055,-0.068664,-0.057875,-0.055864,0.005398,-0.009328,-0.007278


In [160]:
df_scale1 = df_scale[['종목코드', 'time', '영업이익률', '순이익률', 'ROE', '매출액증가율', '총자산증가율',
       '유동자산증가율', '부채비율', '유동부채비율', '비유동부채비율', 'EPS', 'BPS', 'SPS']]
df_scale2 = df_scale1.melt(id_vars = ['종목코드', 'time'])
df_scale2

,종목코드,time,variable,value
0,000020,2016_1분기,영업이익률,0.013698
1,000020,2016_2분기,영업이익률,0.012871
2,000020,2016_3분기,영업이익률,0.012898
3,000020,2016_4분기,영업이익률,0.012602
4,000020,2017_1분기,영업이익률,0.013333
...,...,...,...,...
956779,950220,2022_3분기,SPS,-0.007371
956780,950220,2022_4분기,SPS,-0.007371
956781,950220,2023_1분기,SPS,-0.007371
956782,950220,2023_2분기,SPS,-0.007371


In [161]:
df_scale3 = df_scale2.pivot(index = ['종목코드', 'variable'], columns = "time", values = 'value').reset_index()
df_scale3 = df_scale3.rename_axis(None, axis=1)

df_scale3 = df_scale3.fillna(0)
df_scale3 = df_scale3.rename(columns = {"variable": "지표"})

print(df_scale3.shape)
df_scale3.head(6)

(30864, 33)


,종목코드,지표,2016_1분기,2016_2분기,2016_3분기,2016_4분기,2017_1분기,2017_2분기,2017_3분기,2017_4분기,2018_1분기,2018_2분기,2018_3분기,2018_4분기,2019_1분기,2019_2분기,2019_3분기,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기,2022_2분기,2022_3분기,2022_4분기,2023_1분기,2023_2분기,2023_3분기
0,000020,BPS,-0.009328,-0.009328,-0.009327,-0.009334,-0.009315,-0.009313,-0.009312,-0.009328,-0.009284,-0.009284,-0.009283,-0.009283,-0.009285,-0.009287,-0.009287,-0.009288,-0.009283,-0.009275,-0.009261,-0.009283,-0.009255,-0.009248,-0.009247,-0.009254,-0.009242,-0.009238,-0.009234,-0.009242,-0.009223,-0.009218,0.000000
1,000020,EPS,0.005489,0.005398,0.005409,0.005146,0.005457,0.005451,0.005393,0.004900,0.005487,0.005393,0.005403,0.005334,0.005419,0.005359,0.005380,0.005279,0.005429,0.005536,0.005656,0.005279,0.005507,0.005543,0.005393,0.005300,0.005514,0.005530,0.005538,0.005345,0.005684,0.005556,0.000000
2,000020,ROE,0.017414,0.004805,0.006305,-0.031188,0.012061,0.011186,0.003933,-0.063682,0.013787,0.003515,0.004606,-0.002852,0.006381,-0.000091,0.002185,-0.008972,0.007412,0.018294,0.028970,-0.008727,0.014079,0.017100,0.003299,-0.005486,0.014042,0.015292,0.015818,-0.001096,0.027456,0.016449,0.013470
3,000020,SPS,-0.007283,-0.007278,-0.007286,-0.007415,-0.007281,-0.007271,-0.007275,-0.007421,-0.007256,-0.007253,-0.007259,-0.007427,-0.007260,-0.007259,-0.007260,-0.007433,-0.007271,-0.007267,-0.007272,-0.007422,-0.007264,-0.007258,-0.007268,-0.007428,-0.007244,-0.007241,-0.007247,-0.007434,-0.007223,-0.007237,0.000000
4,000020,매출액증가율,-0.007313,-0.007652,-0.008213,-0.008340,-0.008240,-0.007828,-0.007374,-0.007180,-0.006103,-0.006836,-0.007030,-0.007336,-0.008708,-0.008931,-0.008529,-0.007528,-0.009341,-0.009040,-0.009345,-0.009981,-0.007820,-0.007703,-0.008120,-0.007477,-0.006814,-0.007136,-0.006633,-0.007518,-0.007005,-0.008171,-0.008030
5,000020,부채비율,-0.071277,-0.068664,-0.070275,-0.063612,-0.082564,-0.080409,-0.078065,-0.076600,-0.082457,-0.084417,-0.085937,-0.083764,-0.084380,-0.082081,-0.082493,-0.081985,-0.084306,-0.085139,-0.082139,-0.084099,-0.081761,-0.082572,-0.083090,-0.080859,-0.082229,-0.082903,-0.083986,-0.080368,-0.086649,-0.087824,-0.087602


In [162]:
df_scale3['score'] = df_scale3.iloc[:, 2:].sum(axis=1)

In [163]:
df_scale4 = df_scale3.loc[:, ["종목코드", "지표", "score"]].pivot(index = ['종목코드'], columns = "지표", values = 'score').reset_index()
print(df_scale4.shape)
df_scale4.head(2)

(2572, 13)


지표,종목코드,BPS,EPS,ROE,SPS,매출액증가율,부채비율,비유동부채비율,순이익률,영업이익률,유동부채비율,유동자산증가율,총자산증가율
0,000020,-0.278369,0.162507,0.155763,-0.218992,-0.243274,-2.514506,-1.914183,0.357546,0.407525,-2.191841,0.125710,-0.237741
1,000040,-0.284421,0.161074,-0.822574,-0.220969,-0.165296,3.294879,0.972737,0.262005,0.314928,3.663034,0.117599,-0.285614


In [164]:
# 여기에 변수 중요도 추가
# 안정성 score
df_scale4["안정성"] = - df_scale4["부채비율"] - df_scale4["비유동부채비율"] - df_scale4["유동부채비율"] # 이거 맞나??
# 수익성 score
df_scale4["수익성"] = df_scale4["영업이익률"] + df_scale4["순이익률"] + df_scale4["ROE"]
# 가치성 score
df_scale4["가치성"] = df_scale4["BPS"] + df_scale4["EPS"] + df_scale4["SPS"]
# 성장성 socre
df_scale4["성장성"] = df_scale4["매출액증가율"] + df_scale4["유동자산증가율"] + df_scale4["총자산증가율"]

In [165]:
# type
df_scale4['type'] = df_scale4.loc[:,["안정성", "수익성", "성장성", "가치성"]].idxmax(axis=1)
df_scale4['type_score'] = df_scale4.loc[:,["안정성", "수익성", "성장성", "가치성"]].max(axis=1)

zero_index = df_scale4[(df_scale4.loc[:, ["안정성", "수익성", "성장성", "가치성"]] == 0).all(axis=1)].index
df_scale4.loc[zero_index,"type"] = "No Type"

df_scale4["type"].value_counts()
df_scale4.head()

지표,종목코드,BPS,EPS,ROE,SPS,매출액증가율,부채비율,비유동부채비율,순이익률,영업이익률,유동부채비율,유동자산증가율,총자산증가율,안정성,수익성,가치성,성장성,type,type_score
0,000020,-0.278369,0.162507,0.155763,-0.218992,-0.243274,-2.514506,-1.914183,0.357546,0.407525,-2.191841,0.125710,-0.237741,6.620530,0.920834,-0.334854,-0.355306,안정성,6.620530
1,000040,-0.284421,0.161074,-0.822574,-0.220969,-0.165296,3.294879,0.972737,0.262005,0.314928,3.663034,0.117599,-0.285614,-7.930650,-0.245642,-0.344316,-0.333311,수익성,-0.245642
2,000050,-0.259492,0.167423,0.095993,-0.214302,-0.258257,-0.848399,1.917843,0.349797,0.419005,-1.838554,0.125710,-0.262767,0.769111,0.864795,-0.306371,-0.395314,수익성,0.864795
3,000070,-0.128078,0.210156,0.389311,-0.138770,-0.220304,-0.936191,1.047581,0.315642,0.368062,-1.565005,0.113545,-0.206891,1.453616,1.073015,-0.056691,-0.313650,안정성,1.453616
4,000075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,No Type,0.000000


In [166]:
# 중복 확인= 없음
df_scale_temp = df_scale4[df_scale4.loc[:, ["안정성", "수익성", "성장성", "가치성"]].apply(lambda row: any(row.duplicated()), axis=1)]
df_scale_temp = df_scale_temp[df_scale_temp["type"] != "No Type"].loc[:, ["종목코드","안정성", "수익성", "성장성", "가치성" ,"type", "type_score"]]
df_scale_temp

지표,종목코드,안정성,수익성,성장성,가치성,type,type_score


In [167]:
df_scale4.to_excel(f"{path}/3_1_dartdata_stdscaled_variable_1222.xlsx", index = False)

### 2.4 Melt

### 4_dartdata_melt_1222 :행이 제무제표

In [169]:
df = pd.read_excel(f"{path}/3_dartdata_scaled_variable_1222.xlsx")

In [170]:
df.columns

Index(['종목코드', '종목명', '시장구분', '업종명', 'year', 'quarter', 'time', '영업이익률',
       '순이익률', 'ROE', '매출액증가율', '총자산증가율', '유동자산증가율', '부채비율', '유동부채비율',
       '비유동부채비율', 'EPS', 'BPS', 'SPS', '영업이익률_score', '순이익률_score',
       'ROE_score', '매출액증가율_score', '총자산증가율_score', '유동자산증가율_score',
       'BPS_score', 'SPS_score', 'EPS_score', '부채비율_score', '유동부채비율_score',
       '비유동부채비율_score'],
      dtype='object')

In [171]:
df4 = df[['종목코드', 'time', '영업이익률', '순이익률', 'ROE', '매출액증가율', '총자산증가율',
       '유동자산증가율', '부채비율', '유동부채비율', '비유동부채비율', 'EPS', 'BPS', 'SPS',
           '영업이익률_score', '순이익률_score',
       'ROE_score', '매출액증가율_score', '총자산증가율_score', '유동자산증가율_score',
       'BPS_score', 'SPS_score', 'EPS_score', '부채비율_score', '유동부채비율_score',
       '비유동부채비율_score']]
df_melt = df4.melt(id_vars = ['종목코드', 'time'])
df_melt

,종목코드,time,variable,value
0,000020,2016_1분기,영업이익률,11.080944
1,000020,2016_2분기,영업이익률,3.374031
2,000020,2016_3분기,영업이익률,3.628083
3,000020,2016_4분기,영업이익률,0.866376
4,000020,2017_1분기,영업이익률,7.677175
...,...,...,...,...
1913563,950220,2022_3분기,비유동부채비율_score,20.000000
1913564,950220,2022_4분기,비유동부채비율_score,20.000000
1913565,950220,2023_1분기,비유동부채비율_score,20.000000
1913566,950220,2023_2분기,비유동부채비율_score,20.000000


In [174]:
df_melt2= pd.merge(df_melt, df_code, on = "종목코드", how = "left" )
df_melt2

,종목코드,time,variable,value,종목명,시장구분,업종명
0,000020,2016_1분기,영업이익률,11.080944,동화약품,KOSPI,의약품
1,000020,2016_2분기,영업이익률,3.374031,동화약품,KOSPI,의약품
2,000020,2016_3분기,영업이익률,3.628083,동화약품,KOSPI,의약품
3,000020,2016_4분기,영업이익률,0.866376,동화약품,KOSPI,의약품
4,000020,2017_1분기,영업이익률,7.677175,동화약품,KOSPI,의약품
...,...,...,...,...,...,...,...
1913563,950220,2022_3분기,비유동부채비율_score,20.000000,네오이뮨텍,KOSDAQ,기타서비스
1913564,950220,2022_4분기,비유동부채비율_score,20.000000,네오이뮨텍,KOSDAQ,기타서비스
1913565,950220,2023_1분기,비유동부채비율_score,20.000000,네오이뮨텍,KOSDAQ,기타서비스
1913566,950220,2023_2분기,비유동부채비율_score,20.000000,네오이뮨텍,KOSDAQ,기타서비스


In [173]:
df_melt2.isna().sum()

종목코드             0
time             0
variable         0
value       520506
종목명              0
시장구분             0
업종명              0
dtype: int64

In [175]:
df_melt2.to_csv(f"{path}/4_dartdata_melt_1222.csv", index = False) # 너무 커서 csv

### 2.5 pivot

### 5_dartdata_pivot_1222 : 컬럼이 연도 (지수평활법 안함)

In [176]:
df_pivot = df_melt.pivot(index = ['종목코드', 'variable'], columns = "time", values = 'value').reset_index()
df_pivot = df_pivot.rename_axis(None, axis=1)

df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.rename(columns = {"variable": "지표"})

print(df_pivot.shape)
df_pivot.head(6)

(61728, 33)


,종목코드,지표,2016_1분기,2016_2분기,2016_3분기,2016_4분기,2017_1분기,2017_2분기,2017_3분기,2017_4분기,2018_1분기,2018_2분기,2018_3분기,2018_4분기,2019_1분기,2019_2분기,2019_3분기,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기,2022_2분기,2022_3분기,2022_4분기,2023_1분기,2023_2분기,2023_3분기
0,000020,BPS,8374.108836,8387.306372,8420.235831,8063.220995,9066.058440,9192.367675,9202.567142,8414.902275,10692.823979,10695.847186,10757.832429,10749.495984,10616.938593,10532.401862,10530.100604,10453.777820,10722.212130,11141.145555,11895.849070,10745.263860,12215.746043,12569.369625,12643.095906,12267.377065,12897.033688,13076.562107,13318.413991,12867.391899,13853.822235,14126.997814,0.00000
1,000020,BPS_score,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.00000
2,000020,EPS,171.312963,35.021794,51.477671,-341.071170,122.797196,114.123401,28.065717,-709.013094,168.665008,26.848008,42.158895,-61.392879,65.942586,-22.593719,8.357438,-142.305823,80.876501,240.576144,420.856435,-142.873223,197.290126,252.042982,28.211788,-111.778421,207.692873,231.678004,245.009227,-44.313253,463.050492,271.391474,0.00000
3,000020,EPS_score,15.000000,10.000000,15.000000,5.000000,15.000000,15.000000,10.000000,5.000000,15.000000,10.000000,15.000000,5.000000,15.000000,10.000000,10.000000,5.000000,15.000000,15.000000,20.000000,5.000000,15.000000,20.000000,10.000000,5.000000,15.000000,15.000000,15.000000,5.000000,20.000000,20.000000,0.00000
4,000020,ROE,2.045746,0.417557,0.611357,-4.229962,1.354472,1.241502,0.304977,-8.425684,1.577366,0.251013,0.391890,-0.571123,0.621107,-0.214516,0.079367,-1.361286,0.754289,2.159348,3.537843,-1.329639,1.615048,2.005216,0.223140,-0.911184,1.610393,1.771704,1.839628,-0.344384,3.342402,1.921084,1.53646
5,000020,ROE_score,15.000000,10.000000,10.000000,5.000000,15.000000,15.000000,10.000000,5.000000,15.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,5.000000,10.000000,15.000000,20.000000,5.000000,15.000000,15.000000,10.000000,10.000000,15.000000,15.000000,15.000000,10.000000,20.000000,15.000000,15.00000


In [177]:
4*8 -1  # 기간 2016년 1분기 - 2023년 3분기

31

In [178]:
# 지수 평활법
exp_score = 0
for i in range(2,33):
    exp_score += 0.8 * (0.2 ** (33 - i)) * df_pivot.iloc[:, i]

df_pivot["exp_score"] = exp_score
df_pivot.head(2)

,종목코드,지표,2016_1분기,2016_2분기,2016_3분기,2016_4분기,2017_1분기,2017_2분기,2017_3분기,2017_4분기,2018_1분기,2018_2분기,2018_3분기,2018_4분기,2019_1분기,2019_2분기,2019_3분기,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기,2022_2분기,2022_3분기,2022_4분기,2023_1분기,2023_2분기,2023_3분기,exp_score
0,000020,BPS,8374.108836,8387.306372,8420.235831,8063.220995,9066.05844,9192.367675,9202.567142,8414.902275,10692.823979,10695.847186,10757.832429,10749.495984,10616.938593,10532.401862,10530.100604,10453.77782,10722.21213,11141.145555,11895.84907,10745.26386,12215.746043,12569.369625,12643.095906,12267.377065,12897.033688,13076.562107,13318.413991,12867.391899,13853.822235,14126.997814,0.0,561.44133
1,000020,BPS_score,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.00000,15.000000,15.00000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.0,0.60000


In [179]:
print(0.8 * (0.2 ** (33 - 2)) * df_pivot.iloc[5, 2])
print(0.8 * (0.2 ** (33 - 10)) * df_pivot.iloc[5, 10])
print(0.8 * (0.2 ** (33 - 31)) * df_pivot.iloc[5, 31])
print(0.8 * (0.2 ** (33 - 32)) * df_pivot.iloc[5, 32])

2.5769803776000044e-21
1.0066329600000014e-15
0.4800000000000001
2.4000000000000004


In [180]:
df_pivot['score'] = df_pivot.iloc[:, 2:33].sum(axis=1)

In [181]:
df_pivot2= pd.merge(df_pivot, df_code, on = "종목코드", how = "left" )
df_pivot2.head(2)

,종목코드,지표,2016_1분기,2016_2분기,2016_3분기,2016_4분기,2017_1분기,2017_2분기,2017_3분기,2017_4분기,2018_1분기,2018_2분기,2018_3분기,2018_4분기,2019_1분기,2019_2분기,2019_3분기,2019_4분기,2020_1분기,2020_2분기,2020_3분기,2020_4분기,2021_1분기,2021_2분기,2021_3분기,2021_4분기,2022_1분기,2022_2분기,2022_3분기,2022_4분기,2023_1분기,2023_2분기,2023_3분기,exp_score,score,종목명,시장구분,업종명
0,000020,BPS,8374.108836,8387.306372,8420.235831,8063.220995,9066.05844,9192.367675,9202.567142,8414.902275,10692.823979,10695.847186,10757.832429,10749.495984,10616.938593,10532.401862,10530.100604,10453.77782,10722.21213,11141.145555,11895.84907,10745.26386,12215.746043,12569.369625,12643.095906,12267.377065,12897.033688,13076.562107,13318.413991,12867.391899,13853.822235,14126.997814,0.0,561.44133,328490.267011,동화약품,KOSPI,의약품
1,000020,BPS_score,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.00000,15.000000,15.00000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,0.0,0.60000,450.000000,동화약품,KOSPI,의약품


In [182]:
df_pivot2.to_excel(f"{path}/5_dartdata_pivot_1222.xlsx", index = False)

### 2-6. Scoring

### 6_dartdata_stocktype_1222 : 점수 산출

In [95]:
df_pivot3 = df_pivot.loc[:, ["종목코드", "지표", "score"]].pivot(index = ['종목코드'], columns = "지표", values = 'score').reset_index()
print(df_pivot3.shape)
df_pivot3.head(2)

(2572, 25)


지표,종목코드,BPS,BPS_score,EPS,EPS_score,ROE,ROE_score,SPS,SPS_score,매출액증가율,매출액증가율_score,부채비율,부채비율_score,비유동부채비율,비유동부채비율_score,순이익률,순이익률_score,영업이익률,영업이익률_score,유동부채비율,유동부채비율_score,유동자산증가율,유동자산증가율_score,총자산증가율,총자산증가율_score
0,000020,328490.267011,450.0,1898.105141,370.0,13.825129,370.0,51373.921637,380.0,213.463841,390.0,892.887396,590.0,204.915243,495.0,262.722343,460.0,184.025519,450.0,687.972153,560.0,90.712818,390.0,85.729537,400.0
1,000040,12537.665160,150.0,-243.950634,310.0,-112.501463,280.0,3986.238655,260.0,1108.631877,330.0,6702.786256,180.0,1188.195600,190.0,-993.129447,185.0,-679.622138,195.0,5243.183665,160.0,-85.626829,340.0,-10.324106,340.0


In [96]:
score_columns = ["종목코드",'BPS_score', 'EPS_score','SPS_score',
                 'ROE_score','영업이익률_score', '순이익률_score',
                 '매출액증가율_score','유동자산증가율_score', '총자산증가율_score',
                 '부채비율_score','비유동부채비율_score','유동부채비율_score' ]
df_pivot4 = df_pivot3[score_columns]
df_pivot4

지표,종목코드,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score
0,000020,450.0,370.0,380.0,370.0,450.0,460.0,390.0,390.0,400.0,590.0,495.0,560.0
1,000040,150.0,310.0,260.0,280.0,195.0,185.0,330.0,340.0,340.0,180.0,190.0,160.0
2,000050,585.0,380.0,410.0,340.0,530.0,425.0,340.0,395.0,340.0,335.0,175.0,465.0
3,000070,540.0,490.0,450.0,400.0,410.0,400.0,350.0,410.0,360.0,295.0,170.0,420.0
4,000075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,950170,65.0,120.0,70.0,210.0,220.0,180.0,255.0,225.0,270.0,150.0,145.0,260.0
2568,950190,55.0,110.0,55.0,210.0,240.0,240.0,105.0,80.0,115.0,240.0,240.0,235.0
2569,950200,55.0,110.0,55.0,100.0,80.0,85.0,175.0,100.0,100.0,190.0,180.0,190.0
2570,950210,40.0,80.0,40.0,75.0,0.0,0.0,0.0,95.0,140.0,175.0,150.0,180.0


In [97]:
# 여기에 변수 중요도 추가
# 안정성 score
df_pivot4["안정성"] = df_pivot4["부채비율_score"] + df_pivot4["비유동부채비율_score"] + df_pivot4["유동부채비율_score"]
# 수익성 score
df_pivot4["수익성"] = df_pivot4["영업이익률_score"] + df_pivot4["순이익률_score"] + df_pivot4["ROE_score"]
# 가치성 score
df_pivot4["가치성"] = df_pivot4["BPS_score"] + df_pivot4["EPS_score"] + df_pivot4["SPS_score"]
# 성장성 socre
df_pivot4["성장성"] = df_pivot4["매출액증가율_score"] + df_pivot4["유동자산증가율_score"] + df_pivot4["총자산증가율_score"]

<ipython-input-97-1f3bc0980014>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot4["안정성"] = df_pivot4["부채비율_score"] + df_pivot4["비유동부채비율_score"] + df_pivot4["유동부채비율_score"]
<ipython-input-97-1f3bc0980014>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot4["수익성"] = df_pivot4["영업이익률_score"] + df_pivot4["순이익률_score"] + df_pivot4["ROE_score"]
<ipython-input-97-1f3bc0980014>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [99]:
# type
df_pivot4['type'] = df_pivot4.loc[:,["안정성", "수익성", "성장성", "가치성"]].idxmax(axis=1)
df_pivot4['type_score'] = df_pivot4.loc[:,["안정성", "수익성", "성장성", "가치성"]].max(axis=1)

zero_index = df_pivot4[(df_pivot4.loc[:, ["안정성", "수익성", "성장성", "가치성"]] == 0).all(axis=1)].index
df_pivot4.loc[zero_index,"type"] = "No Type"

df_pivot4["type"].value_counts()

<ipython-input-99-ad6cc4e24f25>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot4['type'] = df_pivot4.loc[:,["안정성", "수익성", "성장성", "가치성"]].idxmax(axis=1)
<ipython-input-99-ad6cc4e24f25>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot4['type_score'] = df_pivot4.loc[:,["안정성", "수익성", "성장성", "가치성"]].max(axis=1)


안정성        1027
성장성         593
수익성         426
가치성         384
No Type     142
Name: type, dtype: int64

In [100]:
df_pivot4.head(2)

지표,종목코드,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score,안정성,수익성,가치성,성장성,type,type_score
0,000020,450.0,370.0,380.0,370.0,450.0,460.0,390.0,390.0,400.0,590.0,495.0,560.0,1645.0,1280.0,1200.0,1180.0,안정성,1645.0
1,000040,150.0,310.0,260.0,280.0,195.0,185.0,330.0,340.0,340.0,180.0,190.0,160.0,530.0,660.0,720.0,1010.0,성장성,1010.0


In [101]:
# 중복 확인
df_temp = df_pivot4[df_pivot4.loc[:, ["안정성", "수익성", "성장성", "가치성"]].apply(lambda row: any(row.duplicated()), axis=1)]
df_temp = df_temp[df_temp["type"] != "No Type"].loc[:, ["종목코드","안정성", "수익성", "성장성", "가치성" ,"type", "type_score"]]
df_temp.head(2)

지표,종목코드,안정성,수익성,성장성,가치성,type,type_score
30,000400,5.0,20.0,5.0,0.0,수익성,20.0
37,000540,5.0,15.0,5.0,0.0,수익성,15.0


In [103]:
duplicate_values_list = []

# 각 행에서 중복된 값을 찾아 리스트에 추가
for index, row in df_temp.iterrows():
    duplicates = row[row.duplicated(keep=False)].tolist()
    if duplicates:
        duplicate_values_list.append(set(duplicates) -set([row["type_score"]]))
    else:
        duplicate_values_list.append(None)
# 데이터프레임에 새로운 컬럼으로 중복된 값 추가
df_temp['duplicate_value'] = duplicate_values_list
df_temp = df_temp[df_temp["duplicate_value"] == set()]
print(df_temp.index)
print(len(df_temp))
df_temp

Int64Index([  76,   95,   96,  114,  208,  316,  518,  582,  587,  658,  672,
             691,  718,  729,  886,  925, 1010, 1013, 1097, 1110, 1112, 1121,
            1202, 1204, 1322, 1371, 1405, 1482, 1509, 1584, 1706, 1722, 1725,
            1738, 1782, 2056, 2193, 2232, 2238, 2303, 2371, 2372, 2397, 2494,
            2521, 2544],
           dtype='int64')
46


지표,종목코드,안정성,수익성,성장성,가치성,type,type_score,duplicate_value
76,001200,5.0,20.0,20.0,0.0,수익성,20.0,{}
95,001500,5.0,25.0,25.0,0.0,수익성,25.0,{}
96,001510,5.0,15.0,15.0,0.0,수익성,15.0,{}
114,001750,5.0,25.0,25.0,0.0,수익성,25.0,{}
208,003540,5.0,20.0,20.0,0.0,수익성,20.0,{}
316,005820,1560.0,1245.0,990.0,1560.0,안정성,1560.0,{}
518,012450,530.0,1210.0,1520.0,1520.0,성장성,1520.0,{}
582,016360,5.0,15.0,15.0,0.0,수익성,15.0,{}
587,016600,20.0,40.0,40.0,0.0,수익성,40.0,{}
658,021880,20.0,10.0,20.0,15.0,안정성,20.0,{}


In [104]:
index_list = list(df_temp.index)
df_pivot4_1 = df_pivot4.loc[~df_pivot4.index.isin(index_list)] # 정상
df_pivot4_2 = df_pivot4.loc[df_pivot4.index.isin(index_list)] # 보정필요
print(df_pivot4.shape)
print(df_pivot4_1.shape)
print(df_pivot4_2.shape)
df_pivot4_2.head(2)

(2572, 19)
(2526, 19)
(46, 19)


지표,종목코드,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score,안정성,수익성,가치성,성장성,type,type_score
76,001200,0.0,0.0,0.0,10.0,0.0,10.0,5.0,0.0,15.0,5.0,0.0,0.0,5.0,20.0,0.0,20.0,수익성,20.0
95,001500,0.0,0.0,0.0,10.0,0.0,15.0,5.0,0.0,20.0,5.0,0.0,0.0,5.0,25.0,0.0,25.0,수익성,25.0


In [128]:
print(len(df_scale4.loc[df_scale4.index.isin(index_list), "type"]))
df_pivot4_2["type"] = df_scale4.loc[df_scale4.index.isin(index_list), "type"]
df_pivot4_2.head(2)

46


<ipython-input-128-a51a3ebe239d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pivot4_2["type"] = df_scale4.loc[df_scale4.index.isin(index_list), "type"]


지표,종목코드,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score,안정성,수익성,가치성,성장성,type,type_score
76,001200,0.0,0.0,0.0,10.0,0.0,10.0,5.0,0.0,15.0,5.0,0.0,0.0,5.0,20.0,0.0,20.0,수익성,20.0
95,001500,0.0,0.0,0.0,10.0,0.0,15.0,5.0,0.0,20.0,5.0,0.0,0.0,5.0,25.0,0.0,25.0,수익성,25.0


In [134]:
df_final = pd.concat([df_pivot4_1, df_pivot4_2], axis = 0).reset_index(drop = True)
print(df_final.shape)
print(df_final["type"].value_counts())

(2572, 19)
안정성        1029
성장성         594
수익성         422
가치성         385
No Type     142
Name: type, dtype: int64


In [142]:
# final_score
df_final["final_score"] = df_final["성장성"] + df_final["안정성"] + df_final["가치성"] + df_final["수익성"]
print(df_final.shape)
df_final

(2572, 20)


지표,종목코드,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score,안정성,수익성,가치성,성장성,type,type_score,final_score
0,000020,450.0,370.0,380.0,370.0,450.0,460.0,390.0,390.0,400.0,590.0,495.0,560.0,1645.0,1280.0,1200.0,1180.0,안정성,1645.0,5305.0
1,000040,150.0,310.0,260.0,280.0,195.0,185.0,330.0,340.0,340.0,180.0,190.0,160.0,530.0,660.0,720.0,1010.0,성장성,1010.0,2920.0
2,000050,585.0,380.0,410.0,340.0,530.0,425.0,340.0,395.0,340.0,335.0,175.0,465.0,975.0,1295.0,1375.0,1075.0,가치성,1375.0,4720.0
3,000070,540.0,490.0,450.0,400.0,410.0,400.0,350.0,410.0,360.0,295.0,170.0,420.0,885.0,1210.0,1480.0,1120.0,가치성,1480.0,4695.0
4,000075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Type,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2567,361390,120.0,140.0,140.0,145.0,155.0,160.0,155.0,150.0,155.0,130.0,140.0,110.0,380.0,460.0,400.0,460.0,안정성,460.0,1700.0
2568,371950,60.0,60.0,55.0,70.0,65.0,75.0,80.0,105.0,120.0,105.0,100.0,100.0,305.0,210.0,175.0,305.0,안정성,305.0,995.0
2569,417500,20.0,30.0,25.0,30.0,30.0,35.0,25.0,15.0,30.0,35.0,30.0,30.0,95.0,95.0,75.0,70.0,안정성,95.0,335.0
2570,432320,20.0,20.0,20.0,40.0,45.0,45.0,20.0,55.0,55.0,30.0,15.0,60.0,105.0,130.0,60.0,130.0,성장성,130.0,425.0


In [139]:
df_final.to_excel(f"{path}/6_dartdata_stocktype_1222.xlsx", index = False)

### 7_dartdata_1222 : 합친 버전(스케일전후)

In [138]:
# merge
df_variable = pd.read_excel(f"{path}/2_dartdata_variable_1215.xlsx")
df_merge = pd.merge(df_variable, df_final, on = "종목코드", how = "left")
print(df_merge.shape)
df_merge.head(2)

(79732, 38)


,종목코드,종목명,시장구분,업종명,year,quarter,time,영업이익률,순이익률,ROE,매출액증가율,총자산증가율,유동자산증가율,부채비율,유동부채비율,비유동부채비율,EPS,BPS,SPS,BPS_score,EPS_score,SPS_score,ROE_score,영업이익률_score,순이익률_score,매출액증가율_score,유동자산증가율_score,총자산증가율_score,부채비율_score,비유동부채비율_score,유동부채비율_score,안정성,수익성,가치성,성장성,type,type_score,final_score
0,000020,동화약품,KOSPI,의약품,2016,1분기,2016_1분기,11.080944,8.078652,2.045746,13.027416,2.235900,4.959643,38.639330,30.390312,8.249018,171.312963,8374.108836,2120.563589,450.0,370.0,380.0,370.0,450.0,460.0,390.0,390.0,400.0,590.0,495.0,560.0,1645.0,1280.0,1200.0,1180.0,안정성,1645.0,5305.0
1,000020,동화약품,KOSPI,의약품,2016,2분기,2016_2분기,3.374031,1.564813,0.417557,9.131184,4.327189,11.058866,41.252657,32.520926,8.731731,35.021794,8387.306372,2238.082339,450.0,370.0,380.0,370.0,450.0,460.0,390.0,390.0,400.0,590.0,495.0,560.0,1645.0,1280.0,1200.0,1180.0,안정성,1645.0,5305.0


In [140]:
df_merge.columns

Index(['종목코드', '종목명', '시장구분', '업종명', 'year', 'quarter', 'time', '영업이익률',
       '순이익률', 'ROE', '매출액증가율', '총자산증가율', '유동자산증가율', '부채비율', '유동부채비율',
       '비유동부채비율', 'EPS', 'BPS', 'SPS', 'BPS_score', 'EPS_score', 'SPS_score',
       'ROE_score', '영업이익률_score', '순이익률_score', '매출액증가율_score',
       '유동자산증가율_score', '총자산증가율_score', '부채비율_score', '비유동부채비율_score',
       '유동부채비율_score', '안정성', '수익성', '가치성', '성장성', 'type', 'type_score',
       'final_score'],
      dtype='object')

In [143]:
df_merge.to_excel(f"{path}/7_dartdata_1222.xlsx", index = False)